In [1]:
import sys
sys.path.append('/home/mila/l/leo.gagnon/latent_control')

%load_ext autoreload
%autoreload 2
from lightning_modules.diffusion_prior import DiffusionPriorTask
import torch
import matplotlib.pyplot as plt
from data.diffusion import KnownLatentDiffusionDataset
from torch2jax import j2t, t2j

/home/mila/l/leo.gagnon/latent_control/venv/lib/python3.10/site-packages/pl_bolts/__init__.py:11: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
/home/mila/l/leo.gagnon/latent_control/venv/lib/python3.10/site-packages/pl_bolts/__init__.py:11: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
/home/mila/l/leo.gagnon/latent_control/venv/lib/python3.10/site-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:34: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  "lr_options": generate_power_seq(LEARNING_RATE_CIFAR, 11),
/home/mila/l/leo.gagnon/late

In [3]:
task = DiffusionPriorTask('mljobdr9')
task.cuda();

Loaded dataset : (11288/1000)
Loaded checkpoing : last.ckpt
Loaded checkpoing : last.ckpt


In [4]:
dataset = task.train_data.dataset 
dataset: KnownLatentDiffusionDataset

In [14]:
raw_latent, latent, cond_input_ids, cond_ignore_mask = dataset[0].values()

/home/mila/l/leo.gagnon/latent_control/data/hmm.py:585: FutureWarning: None encountered in jnp.array(); this is currently treated as NaN. In the future this will result in an error.
  intv_envs = jnp.array(intv_envs)


In [77]:
cond_mask = torch.BoolTensor([True]*200)[None].cuda()

In [78]:
task.diffusion_prior.cfg.sampler = 'ddpm' # ddpm, dpmpp, ddim
task.diffusion_prior.cfg.sampling_timesteps=250

In [76]:
z_t = task.diffusion_prior.sample(100, cond=cond_input_ids, cond_mask=cond_mask, cls_free_guidance=1.0)
z_t = task.diffusion_prior.unnormalize_latent(z_t)

sampling loop time step:   0%|          | 0/500 [00:00<?, ?it/s]

In [87]:
preds = torch.stack(
    [
        torch.Tensor(
            [
                (latent_embds.weight @ z_t[i].T).argmax()
                for latent_embds in task.base_task.model.encoder.latent_embedding
            ]
        )
        for i in range(100)
    ],
    0,
)

In [88]:
true = torch.Tensor(
    [
        (latent_embds.weight @ latent[0].T).argmax()
        for latent_embds in task.base_task.model.encoder.latent_embedding
    ]
)

In [86]:
print(f'Accuracy : {torch.mean(torch.sum(preds == true[None],-1)/len(true))}')

Accuracy : 0.6158332824707031


In [95]:
(torch.sum(preds == true[None],0)/100).mean(0)

tensor(0.6158)

In [93]:
len(preds)

100